# Обработка данных

## Подключение библиотек и считывание данных

In [105]:
# импорт библиотек
import pandas as pd

In [106]:
# считываем данные
data = pd.read_csv('../data/car_price.csv', index_col=0)

data.head()

,Год выпуска,Поколение,Пробег,История пробега,ПТС,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Название,Цена,Обмен
0,2010,I рестайлинг (2009—2012),156 000 км,7 записей в отчёте Автотеки,Оригинал,2,Не битый,2.3 4WD AT (238 л.с.),2.3 л,Бензин,Автомат,Полный,Sport,Внедорожник 5-дверный,Белый,Левый,JMZE*************,Mazda CX-7,1050000,NaN
1,2020,G07 (2018—2022),109 000 км,Проверить в Автотеке,Оригинал,1,Не битый,M50d 3.0 xDrive Steptronic (400 л.с.),3 л,Дизель,Автомат,Полный,Special by Individual,Внедорожник 5-дверный,Синий,Левый,WBAC*************,BMW X7,8800000,Не интересует
2,2023,I (2023—2024),15 км,NaN,NaN,1,Не битый,2.0 T-GDI 4WD DCT (249 л.с.),2 л,Бензин,Робот,Полный,Platinum,Внедорожник 5-дверный,Чёрный,Левый,LVTD*************,EXEED RX,3730000,NaN
3,2024,I (2022—2024),300 км,2 записи в отчёте Автотеки,NaN,1,Не битый,1.5hyb 4WD AT (530 л.с.),1.5 л,Гибрид,Автомат,Полный,Executive,Седан,Чёрный,Левый,LDP9*************,Voyah Passion (Chasing Light),6770000,NaN
4,2024,I (2021—2024),10 км,NaN,Электронный,1,Не битый,2.0 4WD AT (238 л.с.),2 л,Бензин,Автомат,Полный,Flagship,Внедорожник 5-дверный,Белый,Левый,LB37*************,Geely Monjaro,3600000,Не интересует


## Обработка пропущенных значений

### Обзор данных

In [107]:
# удалим дубликаты, пересоберем индексы, посмотрим размерность
data.drop_duplicates(keep='first', inplace=True)
data.reset_index(drop=True, inplace=True)
data.shape

(2109, 20)

In [108]:
# посмотрим на краткое содержание данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Год выпуска           2109 non-null   int64 
 1   Поколение             2109 non-null   object
 2   Пробег                2109 non-null   object
 3   История пробега       2008 non-null   object
 4   ПТС                   1373 non-null   object
 5   Владельцев по ПТС     2109 non-null   object
 6   Состояние             2109 non-null   object
 7   Модификация           2108 non-null   object
 8   Объём двигателя       2107 non-null   object
 9   Тип двигателя         2109 non-null   object
 10  Коробка передач       2109 non-null   object
 11  Привод                2109 non-null   object
 12  Комплектация          1715 non-null   object
 13  Тип кузова            2109 non-null   object
 14  Цвет                  2109 non-null   object
 15  Руль                  2109 non-null   

Видно, что у признаков **История пробега**, **ПТС**, **Модификация**, **Объем двигателя**, **Комплектация** и **Обмен** есть пропущенные значения.

### Признак - **История пробега**

**История пробега** показывает количество информации об автомобиле на конкретном агрегаторе - Авито. Его можно удалить

In [109]:
data.drop(columns="История пробега", inplace=True)

### Признак - **ПТС**

In [110]:
# посмотрим, какие значения может принимать столбец ПТС
data['ПТС'].value_counts()

ПТС
Оригинал       1002
Электронный     213
Дубликат        158
Name: count, dtype: int64

Можно было бы сделать данный признак бинарным - ПТС указан/не указан, однако формат ПТС также может влиять на автомобиля. Считается, что с дубликат ПТС является менее предпочтительным вариантом, так как может, например, указывать на частую смену владельцев. Более подробно про виды ПТС:
- https://rolf-probeg.ru/blog/dublikat-pts-chem-opasen-i-kak-otlichit-ot-originala/?utm_source=google.com&utm_medium=organic&utm_campaign=google.com&utm_referrer=google.com
- https://ufa.tts.ru/blog/sovety-avtomobilistam/vse-ob-elektronnom-pts-v-chem-otlichiya-ot-bumazhnogo-formata-kak-poluchit-epts-i-postavit-mashinu-n/

Данный признак является категориальным. Заполним пропущенные ячейки значением "Не указан"

In [111]:
data['ПТС'].fillna('Не указан', inplace=True)

### Признак - **Модификация**

In [112]:
# посмотрим на некоторые значения 
data['Модификация'].head()

0                    2.3 4WD AT (238 л.с.)
1    M50d 3.0 xDrive Steptronic (400 л.с.)
2             2.0 T-GDI 4WD DCT (249 л.с.)
3                 1.5hyb 4WD AT (530 л.с.)
4                    2.0 4WD AT (238 л.с.)
Name: Модификация, dtype: object

In [113]:
# посмотрим на строки с пропущенными значениями
data[data['Модификация'].isna()]

,Год выпуска,Поколение,Пробег,ПТС,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Название,Цена,Обмен
1621,2014,I (2010—2014),109 000 км,Не указан,2,Не битый,NaN,1.6 л,Бензин,Вариатор,Полный,NaN,Внедорожник 5-дверный,Белый,Левый,SJNF*************,Nissan Juke,1419000,Возможен


Модификацию можно определить по другим признакам. Найдем машину с такими же характеристиками

In [114]:
data[(data['Название'] == 'Nissan Juke') & (data['Привод'] == 'Полный')]

,Год выпуска,Поколение,Пробег,ПТС,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Название,Цена,Обмен
1621,2014,I (2010—2014),109 000 км,Не указан,2,Не битый,NaN,1.6 л,Бензин,Вариатор,Полный,NaN,Внедорожник 5-дверный,Белый,Левый,SJNF*************,Nissan Juke,1419000,Возможен
1667,2011,I (2010—2014),103 000 км,Оригинал,2,Не битый,1.6 CVT (190 л.с.) 4WD,1.6 л,Бензин,Вариатор,Полный,Базовая,Внедорожник 5-дверный,Чёрный,Левый,JN8A*************,Nissan Juke,1170000,Возможен


In [115]:
# запишем вместо пустого значения модификацию - "1.6 CVT (190 л.с.) 4WD"
data.loc[1621, 'Модификация'] = "1.6 CVT (190 л.с.) 4WD"

Модификация включает себя следующий параметры автомобиля: характеристики двигателя (объем, мощность, тип топлива, индекс/наименование), тип привода, тип КПП. Получается, данный признак дублирует информацию 4-х других признаков - **Объём двигателя**, **Тип двигателя**, **Коробка передач**, **Привод**. Тогда заменим признак **Модификация** на **Мощность**, вытащив нужную информацию

In [116]:
data['Мощность'] = data['Модификация'].apply(lambda x: str(x).split()[-2].replace('(', ''))
data.head()

,Год выпуска,Поколение,Пробег,ПТС,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Название,Цена,Обмен,Мощность
0,2010,I рестайлинг (2009—2012),156 000 км,Оригинал,2,Не битый,2.3 4WD AT (238 л.с.),2.3 л,Бензин,Автомат,Полный,Sport,Внедорожник 5-дверный,Белый,Левый,JMZE*************,Mazda CX-7,1050000,NaN,238
1,2020,G07 (2018—2022),109 000 км,Оригинал,1,Не битый,M50d 3.0 xDrive Steptronic (400 л.с.),3 л,Дизель,Автомат,Полный,Special by Individual,Внедорожник 5-дверный,Синий,Левый,WBAC*************,BMW X7,8800000,Не интересует,400
2,2023,I (2023—2024),15 км,Не указан,1,Не битый,2.0 T-GDI 4WD DCT (249 л.с.),2 л,Бензин,Робот,Полный,Platinum,Внедорожник 5-дверный,Чёрный,Левый,LVTD*************,EXEED RX,3730000,NaN,249
3,2024,I (2022—2024),300 км,Не указан,1,Не битый,1.5hyb 4WD AT (530 л.с.),1.5 л,Гибрид,Автомат,Полный,Executive,Седан,Чёрный,Левый,LDP9*************,Voyah Passion (Chasing Light),6770000,NaN,530
4,2024,I (2021—2024),10 км,Электронный,1,Не битый,2.0 4WD AT (238 л.с.),2 л,Бензин,Автомат,Полный,Flagship,Внедорожник 5-дверный,Белый,Левый,LB37*************,Geely Monjaro,3600000,Не интересует,238


In [119]:
# удалим теперь не нужный столбец "Модификация"
data.drop(columns='Модификация', inplace=True)

## Удаление лишних признаков

Уберем лишние признаки. *История пробега* показывает количество информации на конкретном агрегаторе - Авито. *Комплектация* уникальна для каждой модели

In [118]:
data.drop(columns=["История пробега", "Комплектация"], inplace=True)

KeyError: "['История пробега'] not found in axis"

Признак *VIN или номер кузова* можно оставить, если есть Nan значения, и сделать бинарным признаком - VIN указан/не указан

In [ ]:
# проверим, есть ли NaN значения
data['VIN или номер кузова'].isna().any()

False

Данный признак не получится сделать бинарным. Сам по себе номер кузов служит для идентификации конкретного автомобиля. Если бы были видно все символы VIN, можно было бы сделать данный признак индексом. Однако доступны только первые 4 символа и встречаются повторяющиеся значения. Признак *VIN или номер кузова* можно удалить

In [ ]:
data.drop(columns='VIN или номер кузова', inplace=True)

In [ ]:
data['Руль'].value_counts()

Руль
Левый     4923
Правый      23
Name: count, dtype: int64

In [ ]:
data.head()

,Год выпуска,Поколение,Пробег,ПТС,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Тип кузова,Цвет,Руль,Название,Цена,Обмен
0,2010,I рестайлинг (2009—2012),156 000 км,Оригинал,2,Не битый,2.3 4WD AT (238 л.с.),2.3 л,Бензин,Автомат,Полный,Внедорожник 5-дверный,Белый,Левый,Mazda CX-7,1050000,NaN
1,2020,G07 (2018—2022),109 000 км,Оригинал,1,Не битый,M50d 3.0 xDrive Steptronic (400 л.с.),3 л,Дизель,Автомат,Полный,Внедорожник 5-дверный,Синий,Левый,BMW X7,8800000,Не интересует
2,2023,I (2023—2024),15 км,NaN,1,Не битый,2.0 T-GDI 4WD DCT (249 л.с.),2 л,Бензин,Робот,Полный,Внедорожник 5-дверный,Чёрный,Левый,EXEED RX,3730000,NaN
3,2024,I (2022—2024),300 км,NaN,1,Не битый,1.5hyb 4WD AT (530 л.с.),1.5 л,Гибрид,Автомат,Полный,Седан,Чёрный,Левый,Voyah Passion (Chasing Light),6770000,NaN
4,2024,I (2021—2024),10 км,Электронный,1,Не битый,2.0 4WD AT (238 л.с.),2 л,Бензин,Автомат,Полный,Внедорожник 5-дверный,Белый,Левый,Geely Monjaro,3600000,Не интересует
